In [0]:
create or replace MATERIALIZED VIEW bolt_finint_int.silver.fs_max_subscriptions as

with global_subscription_entities as (

-- Comment out the below once we switch to the raw catalogs

-- select tenant, record.subscription.*, 'AMER' as region
-- from bolt_dcp_prod.bronze.raw_s2s_subscription_entities

-- UNION all

-- select tenant, record.subscription.*, 'LATAM' as region
-- from bolt_dcp_prod.beam_latam_bronze.raw_s2s_subscription_entities

-- UNION all 

-- select tenant, record.subscription.*, 'EMEA' as region
-- from bolt_dcp_prod.beam_emea_bronze.raw_s2s_subscription_entities


select tenant, record.subscription.*, 'AMER' as region
from bolt_raw_prod.commerce.raw_beam_amer_s2s_subscription_entities

UNION all

select tenant, record.subscription.*, 'LATAM' as region
from bolt_raw_prod.commerce.raw_beam_latam_s2s_subscription_entities

UNION all 

select tenant, record.subscription.*, 'EMEA' as region
from bolt_raw_prod.commerce.raw_beam_emea_s2s_subscription_entities

UNION ALL

select tenant, record.subscription.*, 'APAC' as region
from bolt_raw_prod.commerce.raw_beam_apac_s2s_subscription_entities

)

select
s.tenant,
s.realm,
s.region,
s.origin,
replace(s.status,'STATUS_','') as status,
s.startedWithFreeTrial,
s.inFreeTrial,
s.subscribedInSite,
s.purchaseTerritory,

case
  when s.direct.paymentProvider is not null then 'Direct'
  when s.iap.provider is not null then 'IAP'
  when s.partner.partnerId is not null then 'Partner'
end as providerType,

coalesce(p.provider,'WEB') as providerName,
direct.subscriptionType as direct_subscriptionType,
s.direct.affiliate as direct_affiliate,

p.pricePlanId,
replace(p.market,'TG_MARKET_','')as pricePlanMarket,
replace(p.producttype,'PRODUCT_TYPE_','') as productType,
p.bundle as isBundle,
p.businessType,
p.businessPlatform as SAPPlatform,
p.businessBrand as SAPBrand,
p.revenueType as SAPRevenueType,
p.businessCase,
p.productname as productName,
replace(p.tiertype,'TIER_TYPE_','') as tierType,
coalesce(direct.campaignCode,iap.campaignCode) as campaignCode,
coalesce(direct.campaignId,iap.campaignId) as campaignId,
p.campaign_name,
p.campaign_type,
p.internalName,
replace(p.priceplantype,'TYPE_','') as pricePlanType,
replace(p.paymentPeriod,'PERIOD_','') as paymentPeriod,
p.numberOfPeriodsBetweenPayments,
p.planprice as planPrice,
p.currency as planCurrency,

s.startDate,
s.direct.affiliateStartDate as affiliateStartDate,
s.nextRenewalDate,
s.nextRetryDate,
s.endDate,
s.cancellationDate,
s.terminationDate,
s.terminationReason,
s.terminationCode,
s.direct.pricePlanChangeDate as pricePlanChangeDate,
s.direct.minimunTermEndDate as minimumTermEndDate,
s.direct.subscriptionInstallment.renewalCycleStartDate as Installment_renewalCycleStartDate,
s.direct.subscriptionInstallment.remainingInstalments as remainingInstallments,
replace(s.direct.subscriptionInstallment.instalmentType, 'TYPE_','') as InstallmentType,
coalesce(s.direct.verifiedHomeTerritory.expirationDate,s.iap.verifiedHomeTerritory.expirationDate) as HomeTerritory_expirationDate,
coalesce(s.direct.verifiedHomeTerritory.code,s.iap.verifiedHomeTerritory.code) as HomeTerritory_code,

p.contractId,
p.fulfillerPartnerId,
p.fulfillerPartnerSku,
s.globalsubscriptionid,
s.baseSubscriptionId,
s.previousSubscriptionGlobalId,
s.userid,
s.direct.paymentMethodId as direct_paymentMethodId,

s.iap.originalProviderPaymentReference as iap_originalProviderPaymentReference,
s.iap.providerUserId as iap_providerUserId,
s.iap.provider as iap_provider,
s.iap.iapSubscriptionId as iap_iapSubscriptionId,
s.iap.pauseDate as iap_pauseDate,
s.iap.pauseCode as iap_pauseCode,
s.iap.pauseReason as iap_pauseReason,

s.partner.gauthSubscriptionId as partner_gauthSubscriptionId,
s.partner.gauthUserId as partner_gauthUserId,
s.partner.partnerId as partner_partnerId,
s.partner.sku as partner_sku,

s.created as recordCreated,
s.updated as recordUpdated,

u.email as userEmail,
u.anonymous,
u.testUser,
u.registeredTimestamp as userRegisteredDate,
u.lastLoginTimestamp as userLastLoginDate

from global_subscription_entities s 
left join bolt_finint_int.silver.fs_max_plans p on s.priceplanid = p.priceplanid
left join bolt_finint_int.silver.fs_max_users u on s.userid = u.userid